In [3]:
from sklearn import datasets
from sklearn import preprocessing
import pandas as pd
import numpy as np
import boto3

Data preprocessing.

In [16]:
iris = datasets.load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns = iris['feature_names']+['target'])
df.rename(columns = {'sepal length (cm)':'sepal_length', 'sepal width (cm)':'sepal_width','petal length (cm)':'petal_length','petal width (cm)':'petal_width'}, inplace = True)
df.target = df.target.astype(int)
df = df.iloc[:, [4,0,1,2,3]]

train, validate, test = np.split(df.sample(frac=1, random_state=42),[int(.6*len(df)), int(.8*len(df))])
train.to_csv('train.csv',index=False)
validate.to_csv('validate.csv',index=False)
test.to_csv('test.csv',index=False)

Upload to S3.

In [18]:
def uploadFileToS3(localFileLocation, remoteFileLocation):
    client = boto3.client('s3')
    s3 = boto3.resource('s3')
    s3.meta.client.upload_file(localFileLocation, 'pengantar-sagemaker', remoteFileLocation)
uploadFileToS3('train.csv','train/train.csv')
uploadFileToS3('validate.csv','validate/validate.csv')
uploadFileToS3('test.csv','test/test.csv')

In [ ]:
# Import necessary modules
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load the Iris dataset
iris = datasets.load_iris()

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)# Train a logistic regression model on the training data
model = LogisticRegression(solver='lbfgs', max_iter=1000, multi_class='auto')
model.fit(X_train, y_train)

# Evaluate the model on the test data
score = model.score(X_test, y_test)

print("Test score: {:.2f}".format(score))